In [ ]:
# import all necessary libraries
import numpy as np 
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from kaggle_datasets import KaggleDatasets
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import models
from sklearn.model_selection import train_test_split

In [ ]:
train_df = pd.read_csv('../input/hotel-id-2021-fgvc8/train.csv')
train_df

In [ ]:
train_df['chain_image'] = '../input/hotel-id-2021-fgvc8/train_images/'+train_df['chain'].astype(str) + '/' + train_df['image']
train_df

In [ ]:
train_df_subset=train_df.loc[train_df['chain']==90]
train_df_subset['hotel_id']=train_df_subset['hotel_id'].astype(str)
train_df_subset

In [ ]:
X_train, X_test = train_test_split(train_df_subset,
                                  stratify=train_df_subset['hotel_id'],
                                  test_size=0.1,
                                  random_state=11)

In [ ]:
# # detect and init the TPU
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# # instantiate a distribution strategy
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

# DIR = KaggleDatasets().get_gcs_path()
# Train_PATH = DIR + "/train_images/90"

In [ ]:
filepath='../input/hotel-id-2021-fgvc8/train_images/90'


training=ImageDataGenerator(rescale = 1./255,
                           shear_range=0.2,
                           zoom_range=0.2,
                           horizontal_flip=True,
                           rotation_range=0.1).flow_from_dataframe(
    dataframe=X_train,
    directory=filepath,
    x_col="image",
    y_col="hotel_id",
    weight_col=None,
    #target_size=(256, 256),
    target_size=(224, 224),
    color_mode="rgb",
    classes=None,
    class_mode="categorical",
    batch_size=32,
    shuffle=True,
    seed=11,
    save_to_dir=None,
    save_prefix="",
    save_format="png",
    subset=None,
    interpolation="nearest",
    validate_filenames=True
)

testing=ImageDataGenerator().flow_from_dataframe(
    dataframe=X_test,
    directory=filepath,
    x_col="image",
    y_col="hotel_id",
    weight_col=None,
    #target_size=(256, 256),
    target_size=(224, 224),
    color_mode="rgb",
    classes=None,
    class_mode="categorical",
    batch_size=32,
    shuffle=True,
    seed=11,
    save_to_dir=None,
    save_prefix="",
    save_format="png",
    subset=None,
    interpolation="nearest",
    validate_filenames=True
)

In [ ]:
training.class_indices

In [ ]:
testing.class_indices

In [ ]:
# model = tf.keras.applications.EfficientNetB0(
#     include_top=True,
#     weights=None,
#     input_tensor=None,
#     input_shape=None,
#     pooling=None,
#     classes=270,
#     classifier_activation="softmax"
# )

EffNet = EfficientNetB0(include_top=False,
                             weights='imagenet',
                             input_tensor=None,
                             input_shape=([224, 224, 3]),
                             pooling='avg'
                       )

model_EffNet = models.Sequential()
model_EffNet.add(EffNet)
model_EffNet.add(Dropout(0.5))
model_EffNet.add(Dense(270, activation='softmax'))


model_EffNet.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [ ]:
model_EffNet.fit(training, callbacks=[early_stop], epochs=25)

In [ ]:
predictions = model_EffNet.predict_classes(testing)

In [ ]:
pred = np.argmax(model_EffNet.predict(testing), axis=-1)

In [ ]:
predictions

In [ ]:
pred

In [ ]:
testing.class_indices

In [ ]:
testing[0][0][0]

In [ ]:
model_EffNet.save('model_EffNetv2.h5')

In [ ]:
load_

In [ ]:
# f = open("listouput.txt",mode='x')
# f.write(hotelLabels)
# f.close()

import pickle

with open('listOutput.pickel', 'x') as f:
    pickle.dump(hotelLabels, f)